# Add FAA Data to a Voxel 51 Dataset
adapted from: https://voxel51.com/docs/fiftyone/recipes/model_inference.html

If you haven’t already, install FiftyOne:

# Build FiftyOne Dataset

In [ ]:
#config
import fiftyone as fo
import os
import pandas as pd

planes = pd.read_csv("./aircraftDatabase.csv",index_col='icao24')
#print(planes)
batch_size = 1000
# Create the dataset
dataset = fo.load_dataset("plane-dataset")

    


In [ ]:


def addPlaneData(sample):

    plane_id = sample["icao24"].label.lower()
    try:
        plane = planes.loc[ plane_id.lower()]
        if plane.size == 26:

            #print("Adding metadata for plane {} onto image {}".format(plane["icao24"].values[0], image["external_id"]))
            if isinstance(plane["model"], str):
                sample["model"] = fo.Classification(label=plane["model"] )
            if isinstance(plane["manufacturername"], str):
                sample["manufacturer"] = fo.Classification(label=plane["manufacturername"])
            #print(sample)
            sample.save()
        else:
            print(plane.size)
    except KeyError:
        print("FAA Data entry not found for: {}".format(plane_id))        


In [ ]:
#sample=dataset.shuffle().first()
for sample in dataset:
    addPlaneData(sample)

### Find unique models
Go through the models that were attached to samples and create a list of all of the unique model types

In [ ]:
models = []
view = dataset.exists("model")

for s in view.select_fields("model"):
    if s.model.label not in models:
        models.append(s.model.label)
for model in models:
    print(model)

### Normalize the Model
There are a lot of different types of models and different ways of writing the same models. We need to try and group together very similar aircraft types. This creates a new field with the normalized model call: `norm_model`

In [ ]:
def normalizeModel(model):
    norm_model=None
    if model=="A319-112" or model=="A319-115" or model=="A319-132":
        norm_model="A319"

    if model=="A320 232" or model=="A320 232SL" or model=="A320-211" or model=="A320-212" or model=="A320-214" or model=="A320-232" or model=="A320-251N" or model=="A320-271N":
        norm_model="A320"

    if model=="A321-211" or model=="A321-231" or model=="A321-271NX" or model=="Airbus A321-231(SL)":
        norm_model="A321"

    if model=="A330 243" or model=="A330-243F":
        norm_model="A330"

    if model=="737-71B" or model=="737-724" or model=="737-73V" or model=="737-752" or model=="737-7H4" or model=="737-7Q8":
        norm_model="737-700"

    if model=="737-800" or model=="Boeing 737-852" or model=="737-823" or model=="737-824" or model=="737-832" or model=="737-83N" or model=="737-84P" or model=="737-890" or model=="737-8EH" or model=="737-8H4" or model=="737NG 823/W" or model=="737NG 852/W" or model=="737NG 85P/W" or model=="737NG 86N/W" or model=="737NG 8V3/W":
        norm_model="737-800"

    if model=="737-900ER" or model=="737-924ER" or model=="737-932ER":
        norm_model="737-900"


    if model=="747-48EF":
        norm_model="747-400"

    if model=="757-231" or model=="757-232" or model=="757-251":
        norm_model="757-200"

    if model=="767 330ER/W":
        norm_model="767-300"

    if model=="777-223":
        norm_model="777-200"

    if model=="787-8":
        norm_model="787-800"

    if model=="787-9 (Boeing)" or model=="BOEING 787-9 Dreamliner":
        norm_model="787-800"

    if model=="45" or model=="60":
        norm_model="Learjet 45/60"

    if model=="510" or model=="Citation Excel" or model=="Citation Sovereign+" or model=="525" or model=="550" or model=="560" or model=="680" or model=="750" or model=="525A" or model=="525B" or model=="525C" or model=="560XL" or model=="680A":
        norm_model="Cessna Jet"

    if model=="CL-600-2B16" or model=="BD-100-1A10" or model=="BD-700-1A11":
        norm_model="Bombardier Challanger"


    if model=="CL-600-2C10":
        norm_model="CRJ700"

    if model=="CL-600-2C11":
        norm_model="CRJ550"

    if model=="CL-600-2D24" or model=="CRJ 900 LR NG" or model=="CRJ-900":
        norm_model="CRJ900"

    if model=="ERJ 170-100 SE" or model=="ERJ 170-100SU" or model=="ERJ 170-200 LR" or model=="ERJ 190-100 IGW" or model=="EMB-190 AR":
        norm_model="ERJ-170"

    if model=="EMB-135BJ" or model=="EMB-145LR":
        norm_model="EMB-135"

    if model=="EMB-505" or model=="EMB-545":
        norm_model="EMB-505"

    if model=="PA-23-250":
        norm_model="Piper PA-23"

    if model=="PC-12/47E":
        norm_model="Pilatus PC-12"
        
    if model=="FALCON 10" or model=="FALCON 2000" or model=="FALCON 50" or model=="FALCON 7X" or model=="FALCON 900 EX" or model=="FALCON 900EX":    
        norm_model="Falcon"
    
    if model=="G-IV" or model=="G-V" or model=="GALAXY" or model=="GIV-X (G450)" or model=="GULFSTREAM 200" or model=="GV-SP (G550)" or model=="GVI(G650ER)":
        norm_model="Gulfstream"
        
    if model=="HAWKER 800XP" or model=="HAWKER 900XP":
        norm_model="Hawker"
    
    if model=="SF50":
        norm_model="Cirrus"
    
    if model=="PRESSURIZED LANCR IV":
        norm_model="Lancair IV"
        
    if model=="B300":
        norm_model="King Air"
    return norm_model


### Add the normalized label for the model
This groups to together similar models and different names for the same model

In [ ]:
for sample in dataset.exists("model"):
    norm_model = normalizeModel(sample["model"].label)
    if norm_model != None:
        sample["norm_model"] = fo.Classification(label=norm_model)
        sample.save()
    else:
        print("Match not found for: {}".format(sample["model"].label))

    #addPlaneData(sample)